In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import open_clip
from torchvision import datasets, transforms
import torch
from PIL import Image
import open_clip
from tqdm import tqdm
from torch.utils.data import DataLoader
import numpy as np
from sklearn import calibration
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch.nn as nn
import torch.optim as optim
from sklearn.tree import DecisionTreeRegressor as DT
from sklearn.isotonic import IsotonicRegression as IR
from scipy.stats import norm
from tqdm import tqdm
import pandas as pd
import json

/home/ubuntu/anaconda3/envs/content_understanding/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from util import get_preds, calc_bins, get_metrics, T_scaling, find_temp_scale, get_openai_prompts, get_val_set, find_temp_scale_with_q, get_text_probs, sample_quantile

In [4]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
batch_size = 64

In [5]:
models_to_consider = [
    #('ViT-B-32' , 'laion2b_s34b_b79k'),
    ('ViT-L-14' , 'laion2b_s32b_b82k'),
    ('ViT-H-14' , 'laion2b_s32b_b79k'),
    #('ViT-bigG-14' , 'laion2b_s39b_b160k'),
    #('ViT-B-32', 'openai')
]

In [7]:
def do_full_run():
    dataset_name = 'CIFAR100'
    num_classes = 100
    cifar_test = datasets.CIFAR100('/home/ubuntu/data/', train = False, transform = preprocess, download=True)
    classes, templates = get_openai_prompts(dataset_name)
    cifar_test.classes = classes
    val_dset = get_val_set(dataset_name, classes, preprocess)
    # No scaling
    eces = []
    mces = []
    accs = []
    for t in tqdm(templates):
        predictions, actual, probs = get_preds(model, tokenizer, cifar_test, text_template=t, device=device)
        ece, mce, acc = get_metrics(predictions, actual, probs)
        eces.append(ece)
        accs.append(acc)
        mces.append(mce)

    # With scaling
    eces_scaled = []
    mces_scaled = []
    accs_scaled = []
    temps = []
    for t in tqdm(templates):
        scaled_temp = find_temp_scale(model, tokenizer, val_dset, num_classes=num_classes, text_template=t, device=device)
        predictions, actual, probs = get_preds(model, tokenizer, cifar_test, text_template=t, temp_scaling=scaled_temp, device=device)
        ece, mce, acc = get_metrics(predictions, actual, probs)
        eces_scaled.append(ece)
        accs_scaled.append(acc)
        mces_scaled.append(mce)
        temps.append(scaled_temp)

    with open(f'numbers/q1.0/{dataset_name}_{model_name}_{pretrained_dset}_acrossprompt.json', 'w') as f:
        json.dump({
            'eces_scaled' : eces_scaled,
            'mces_scaled' : mces_scaled,
            'accs_scaled' : accs_scaled,
            'eces' : eces,
            'mces' : mces,
            'accs' : accs,
            'temps' : temps
        },f)

In [8]:
for model_name, pretrained_dset in models_to_consider:
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained_dset, device=device)
    tokenizer = open_clip.get_tokenizer(model_name)
    do_full_run()
    

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 19/19 [33:43<00:00, 106.49s/it]


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 19/19 [58:17<00:00, 184.08s/it]
